In [1]:
# Authors:Haben Abaye & Payal Singh
# Course: CS697A
# Date: 12/27/2020

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import warnings
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from collections import Counter
from collections import defaultdict
from scipy.special import boxcox1p
from sklearn.preprocessing import RobustScaler
from sklearn import metrics
from scipy.stats import norm,skew,probplot
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor,StackingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor

In [2]:
def get_outliers(data,features): 
    all_outliers = defaultdict(list)
    outlier_array = []
    outlier_number = 5
    for col in features:
        lower_bound = np.percentile(data[col], 25)
        upper_bound = np.percentile(data[col], 75)       
        IQR = upper_bound - lower_bound 
        outlier_range = 1.7 * IQR        
        outlier = data[(data[col] < lower_bound - outlier_range) | (data[col] > upper_bound + outlier_range )]
        outlier_index = outlier.index        
        outlier_array.extend(outlier_index)
        
    outlier_array = Counter(outlier_array) 
    for key,value in outlier_array.items():
        if(value > outlier_number):
            all_outliers[key].append(value)
    return all_outliers
    
def clean_data(data):
    threshold_value = 0.8
    data = data[data.columns[data.isnull().mean() < threshold_value]]
    return data

def handle_missing_value(data):
    for col in data.select_dtypes(include=['int64', 'float64']):
        data[col] = data[col].fillna(data[col].median())
        
    object_column = data.loc[:,'MSSubClass':'SaleCondition'].select_dtypes(include='object').columns
    for col in object_column:
        data[col] = data[col].fillna('NA')
    return data

def encode_data(data):
    object_column = data.loc[:,'MSSubClass':'SaleCondition'].select_dtypes(include='object').columns
    #handle ordinal data
    ordinal = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond' ,'HeatingQC', 'KitchenQual', 
               'FireplaceQu','GarageQual', 'GarageCond','Utilities','BsmtFinType1','BsmtFinType2',
               'GarageFinish']
    label_mapping = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0,
                     'AllPub':4, 'NoSewr':3, 'NoSeWa':2, 'ELO':1,
                    'Av':3, 'Mn':2, 'No':1, 'GLQ':6, 'ALQ':5, 'BLQ':4,
                    'Rec':3, 'LwQ':2, 'Unf':1, 'Y':2, 'N':1, 'Fin':3,
                    'RFn': 2, 'GdPrv':4, 'MnPrv':3, 'GdWo':2, 'MnWw':1}
    for col in ordinal:
        data[col] = data[col].map(label_mapping)
        
    #handle nominal data
    nominal  = [item for item in object_column if item not in ordinal]
    data = pd.get_dummies(data, columns=nominal, drop_first=True)
    return data

def add_features(data):
    data['Total_area'] = data['LotArea']+data['1stFlrSF']+ data['2ndFlrSF']
    data['Overall_type'] = data['OverallQual'] + data['OverallCond']
    data['age']= abs(data['YrSold'] - data['YearBuilt'] )
    data['remodeled'] = data['YearRemodAdd'] - data['YearBuilt']
    return data


def handle_skewness(data):
    numerical_value= data.select_dtypes(exclude=object).columns
    skew_value=data[numerical_value].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
    max_skew = 0.60
    skew_value=skew_value[abs(skew_value)>max_skew]
    skew_columns = skew_value.index
    for col in skew_columns:
        data[col] = boxcox1p(data[col], .22)
    return data

In [5]:
#load training data
url = "train.csv"
train_csv = pd.read_csv(url)
train = pd.DataFrame(train_csv)
# Load Testing dataset
url = "test.csv"
test_csv = pd.read_csv(url)
test = pd.DataFrame(test_csv)

In [6]:
train['SalePrice'] = np.log1p(train['SalePrice'])

In [7]:
#find outliers of training from numeric column 
numeric_data = train.loc[:,'MSSubClass':'SaleCondition'].select_dtypes(exclude=['object'])
numeric_column = numeric_data.columns

#get outliers
outliers = get_outliers(train, numeric_column)
#drop outliers
train = train.drop(outliers, axis = 0).reset_index(drop=True)

total_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'], test.loc[:,'MSSubClass':'SaleCondition']))
total_data = add_features(total_data)

total_data = clean_data(total_data)
total_data = handle_missing_value(total_data)
total_data = encode_data(total_data)
total_data = handle_skewness(total_data)

In [8]:
X_train_data = total_data[:train.shape[0]]
X_test_house = total_data[train.shape[0]:]
label_data = train.SalePrice

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_train_data, label_data, test_size=0.07,random_state=42)

In [10]:
rf = RandomForestRegressor(bootstrap=True,criterion='mse',
                           random_state=42,max_depth=45,
                           max_features=30,
                           n_estimators=1500,
                           n_jobs=-1, verbose=2)
rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 1500building tree 2 of 1500

building tree 3 of 1500
building tree 4 of 1500
building tree 5 of 1500
building tree 6 of 1500
building tree 7 of 1500building tree 8 of 1500

building tree 9 of 1500building tree 10 of 1500
building tree 11 of 1500building tree 12 of 1500

building tree 13 of 1500

building tree 14 of 1500
building tree 15 of 1500
building tree 16 of 1500
building tree 17 of 1500
building tree 18 of 1500
building tree 19 of 1500
building tree 20 of 1500building tree 21 of 1500

building tree 22 of 1500
building tree 23 of 1500building tree 24 of 1500

building tree 25 of 1500
building tree 26 of 1500
building tree 27 of 1500building tree 28 of 1500

building tree 29 of 1500
building tree 30 of 1500
building tree 31 of 1500
building tree 32 of 1500
building tree 33 of 1500
building tree 34 of 1500building tree 35 of 1500
building tree 36 of 1500

building tree 37 of 1500
building tree 38 of 1500
building tree 39 of 1500
building tree 40 of 1500
building 

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.1s


building tree 71 of 1500building tree 72 of 1500

building tree 73 of 1500
building tree 74 of 1500
building tree 75 of 1500
building tree 76 of 1500
building tree 77 of 1500
building tree 78 of 1500
building tree 79 of 1500
building tree 80 of 1500building tree 81 of 1500
building tree 82 of 1500

building tree 83 of 1500
building tree 84 of 1500
building tree 85 of 1500building tree 86 of 1500
building tree 87 of 1500

building tree 88 of 1500building tree 89 of 1500building tree 90 of 1500


building tree 91 of 1500
building tree 92 of 1500building tree 93 of 1500

building tree 94 of 1500building tree 95 of 1500

building tree 96 of 1500
building tree 97 of 1500building tree 98 of 1500

building tree 99 of 1500building tree 100 of 1500building tree 101 of 1500


building tree 102 of 1500building tree 103 of 1500
building tree 104 of 1500

building tree 105 of 1500building tree 106 of 1500

building tree 107 of 1500building tree 108 of 1500

building tree 109 of 1500
building tree 1

[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    0.7s


...

building tree 1348 of 1500building tree 1349 of 1500building tree 1350 of 1500building tree 1351 of 1500building tree 1352 of 1500
building tree 1353 of 1500




building tree 1354 of 1500
building tree 1355 of 1500
building tree 1356 of 1500
building tree 1357 of 1500
building tree 1358 of 1500
building tree 1359 of 1500
building tree 1360 of 1500
building tree 1361 of 1500
building tree 1362 of 1500
building tree 1363 of 1500
building tree 1364 of 1500
building tree 1365 of 1500
building tree 1366 of 1500building tree 1367 of 1500

building tree 1368 of 1500
building tree 1369 of 1500
building tree 1370 of 1500
building tree 1371 of 1500building tree 1372 of 1500

building tree 1373 of 1500
building tree 1374 of 1500
building tree 1375 of 1500
building tree 1376 of 1500
building tree 1377 of 1500
building tree 1378 of 1500building tree 1379 of 1500

building tree 1380 of 1500
building tree 1381 of 1500
building tree 1382 of 1500
building tree 1383 of 1500building tree 1384 of 1500

b

[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:    7.4s


building tree 1452 of 1500

building tree 1453 of 1500
building tree 1454 of 1500
building tree 1455 of 1500
building tree 1456 of 1500building tree 1457 of 1500

building tree 1458 of 1500
building tree 1459 of 1500building tree 1460 of 1500

building tree 1461 of 1500
building tree 1462 of 1500building tree 1463 of 1500

building tree 1464 of 1500
building tree 1465 of 1500
building tree 1466 of 1500
building tree 1467 of 1500
building tree 1468 of 1500
building tree 1469 of 1500building tree 1470 of 1500
building tree 1471 of 1500

building tree 1472 of 1500building tree 1473 of 1500

building tree 1474 of 1500
building tree 1475 of 1500
building tree 1476 of 1500
building tree 1477 of 1500
building tree 1478 of 1500
building tree 1479 of 1500building tree 1480 of 1500
building tree 1481 of 1500

building tree 1482 of 1500
building tree 1483 of 1500
building tree 1484 of 1500
building tree 1485 of 1500building tree 1486 of 1500building tree 1487 of 1500

building tree 1488 of 1500



[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    7.7s finished


RandomForestRegressor(max_depth=45, max_features=30, n_estimators=1500,
                      n_jobs=-1, random_state=42, verbose=2)

In [11]:
xgb = XGBRegressor(booster='gbtree',
                   colsample_bylevel=0.6, colsample_bynode=.4,#1,.5
                   colsample_bytree=0.6, gamma=0,
                   importance_type='gain', learning_rate=0.01,
                   max_delta_step=0, max_depth=45,
                   min_child_weight=4, n_estimators=2500,
                   n_jobs=-1, nthread=None,
                   objective='reg:linear', reg_alpha=0.6,
                   reg_lambda=0.6, scale_pos_weight=1,
                   silent=None, subsample=0.8,
                   verbosity=2)
xgb.fit(X_train, y_train)

[09:23:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 0 extra nodes, 0

[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned

[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 4 extra nodes, 0 pruned nodes, max_depth=2
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned

[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[09:23:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned

[09:23:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[09:23:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=3
[09:23:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[09:23:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[09:23:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[09:23:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 10 extra nodes, 0 prune

...

[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=14
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=15
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 52 extra nodes, 0 pruned nodes, max_depth=11
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=11
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 56 extra nodes, 0 pruned nodes, max_depth=14
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 62 extra node

[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 74 extra nodes, 0 pruned nodes, max_depth=13
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 70 extra nodes, 0 pruned nodes, max_depth=11
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=7
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=13
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 86 extra nodes, 0 pruned nodes, max_depth=19
[09:23:23] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 92 extra nodes

[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 78 extra nodes, 0 pruned nodes, max_depth=19
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 66 extra nodes, 0 pruned nodes, max_depth=11
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 72 extra nodes, 0 pruned nodes, max_depth=16
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 58 extra nodes, 0 pruned nodes, max_depth=16
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 52 extra nodes,

[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 66 extra nodes, 0 pruned nodes, max_depth=17
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=23
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 70 extra nodes, 0 pruned nodes, max_depth=13
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 90 extra nodes, 0 pruned nodes, max_depth=15
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=12
[09:23:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 10 extra node

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.6,
             colsample_bynode=0.4, colsample_bytree=0.6, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=45,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=2500, n_jobs=-1, nthread=-1, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0.6,
             reg_lambda=0.6, scale_pos_weight=1, silent=None, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=2)

In [12]:
lgb = LGBMRegressor(max_depth=45, num_leaves=50,
                    learning_rate=0.05, n_jobs=-1,
                    boosting_type='gbdt', objective='regression',
                    metric='rmse', verbosity=2,
                    bagging_fraction=0.7, feature_fraction=0.6,
                    bagging_freq=4, bagging_seed=42,
                    seed=42, colsample_bynode=.6,
                    colsample_bytree=.6)
lgb.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.926138
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.614145
[LightGBM] [Debug] init for col-wise cost 0.004646 seconds, init 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 36 and max_depth = 16
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 36 and max_depth = 12
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 35 and max_depth = 16
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 36 and max_depth = 18
[LightGBM] [Debug] Re-bagging, using 951 data to train
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 38 and max_depth = 17
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 38 and max_depth = 14
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMRegressor(bagging_fraction=0.7, bagging_freq=4, bagging_seed=42,
              colsample_bynode=0.6, colsample_bytree=0.6, feature_fraction=0.6,
              learning_rate=0.05, max_depth=45, metric='rmse', num_leaves=50,
              objective='regression', seed=42, verbosity=2)

In [13]:
knn = KNeighborsRegressor(n_neighbors=4,leaf_size=60,
                          p=1,metric='manhattan',weights='distance')
knn.fit(X_train, y_train)

KNeighborsRegressor(leaf_size=60, metric='manhattan', n_neighbors=4, p=1,
                    weights='distance')

In [14]:
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003))
svr.fit(X_train, y_train)

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('svr', SVR(C=20, epsilon=0.008, gamma=0.0003))])

In [15]:
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.01,
                                max_depth=4, max_features='sqrt',
                                min_samples_leaf=15, min_samples_split=5,
                                loss='huber', random_state =42) 
gbr.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, loss='huber', max_depth=4,
                          max_features='sqrt', min_samples_leaf=15,
                          min_samples_split=5, n_estimators=3000,
                          random_state=42)

In [16]:
dtr = DecisionTreeRegressor(criterion='mse',random_state=42,
                            max_depth=5, max_features='sqrt',
                            min_samples_leaf=15, min_samples_split=10)

In [17]:
abr = AdaBoostRegressor(dtr,n_estimators=800,
                        random_state=42,learning_rate=0.5)
abr.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5,
                                                       max_features='sqrt',
                                                       min_samples_leaf=15,
                                                       min_samples_split=10,
                                                       random_state=42),
                  learning_rate=0.5, n_estimators=800, random_state=42)

In [18]:
cat = CatBoostRegressor(depth=6,learning_rate=0.1,
                        n_estimators=3500,eval_metric = 'RMSE')
cat.fit(X_train, y_train)

0:	learn: 0.3710713	total: 226ms	remaining: 13m 12s
1:	learn: 0.3483111	total: 242ms	remaining: 7m 3s
2:	learn: 0.3277989	total: 253ms	remaining: 4m 55s
3:	learn: 0.3073928	total: 262ms	remaining: 3m 49s
4:	learn: 0.2895026	total: 271ms	remaining: 3m 9s
5:	learn: 0.2752441	total: 290ms	remaining: 2m 48s
6:	learn: 0.2604038	total: 299ms	remaining: 2m 29s
7:	learn: 0.2478638	total: 306ms	remaining: 2m 13s
8:	learn: 0.2356149	total: 316ms	remaining: 2m 2s
9:	learn: 0.2255169	total: 331ms	remaining: 1m 55s
10:	learn: 0.2161057	total: 338ms	remaining: 1m 47s
11:	learn: 0.2064950	total: 345ms	remaining: 1m 40s
12:	learn: 0.1987087	total: 352ms	remaining: 1m 34s
13:	learn: 0.1904455	total: 361ms	remaining: 1m 29s
14:	learn: 0.1839329	total: 368ms	remaining: 1m 25s
15:	learn: 0.1777122	total: 376ms	remaining: 1m 21s
16:	learn: 0.1718507	total: 383ms	remaining: 1m 18s
17:	learn: 0.1672354	total: 391ms	remaining: 1m 15s
18:	learn: 0.1625863	total: 398ms	remaining: 1m 12s
19:	learn: 0.1574824	tot

162:	learn: 0.0722043	total: 1.64s	remaining: 33.5s
163:	learn: 0.0720492	total: 1.64s	remaining: 33.4s
164:	learn: 0.0719090	total: 1.65s	remaining: 33.3s
165:	learn: 0.0715397	total: 1.66s	remaining: 33.3s
166:	learn: 0.0711771	total: 1.66s	remaining: 33.2s
167:	learn: 0.0708055	total: 1.67s	remaining: 33.2s
168:	learn: 0.0706519	total: 1.68s	remaining: 33.2s
169:	learn: 0.0706224	total: 1.69s	remaining: 33.1s
170:	learn: 0.0705524	total: 1.7s	remaining: 33s
171:	learn: 0.0704624	total: 1.7s	remaining: 33s
172:	learn: 0.0701415	total: 1.71s	remaining: 32.9s
173:	learn: 0.0701146	total: 1.72s	remaining: 32.9s
174:	learn: 0.0698926	total: 1.73s	remaining: 32.8s
175:	learn: 0.0698673	total: 1.74s	remaining: 32.8s
176:	learn: 0.0696340	total: 1.75s	remaining: 32.8s
177:	learn: 0.0693454	total: 1.76s	remaining: 32.8s
178:	learn: 0.0691141	total: 1.76s	remaining: 32.7s
179:	learn: 0.0690555	total: 1.77s	remaining: 32.7s
180:	learn: 0.0688679	total: 1.78s	remaining: 32.6s
181:	learn: 0.0687

333:	learn: 0.0466291	total: 3.03s	remaining: 28.7s
334:	learn: 0.0465628	total: 3.04s	remaining: 28.7s
335:	learn: 0.0465530	total: 3.04s	remaining: 28.7s
336:	learn: 0.0464699	total: 3.05s	remaining: 28.6s
337:	learn: 0.0464227	total: 3.06s	remaining: 28.6s
338:	learn: 0.0464126	total: 3.06s	remaining: 28.6s
339:	learn: 0.0462847	total: 3.08s	remaining: 28.6s
340:	learn: 0.0461709	total: 3.08s	remaining: 28.6s
341:	learn: 0.0460273	total: 3.09s	remaining: 28.6s
342:	learn: 0.0458520	total: 3.1s	remaining: 28.6s
343:	learn: 0.0457188	total: 3.11s	remaining: 28.6s
344:	learn: 0.0456013	total: 3.12s	remaining: 28.5s
345:	learn: 0.0455005	total: 3.13s	remaining: 28.5s
346:	learn: 0.0453612	total: 3.14s	remaining: 28.5s
347:	learn: 0.0452921	total: 3.15s	remaining: 28.5s
348:	learn: 0.0452288	total: 3.16s	remaining: 28.5s
349:	learn: 0.0450396	total: 3.17s	remaining: 28.5s
350:	learn: 0.0449456	total: 3.17s	remaining: 28.5s
351:	learn: 0.0449372	total: 3.18s	remaining: 28.5s
352:	learn: 0

494:	learn: 0.0333895	total: 4.43s	remaining: 26.9s
495:	learn: 0.0332781	total: 4.44s	remaining: 26.9s
496:	learn: 0.0331345	total: 4.45s	remaining: 26.9s
497:	learn: 0.0329965	total: 4.46s	remaining: 26.9s
498:	learn: 0.0329727	total: 4.47s	remaining: 26.9s
499:	learn: 0.0329687	total: 4.48s	remaining: 26.9s
500:	learn: 0.0329137	total: 4.48s	remaining: 26.8s
501:	learn: 0.0328265	total: 4.49s	remaining: 26.8s
502:	learn: 0.0327420	total: 4.5s	remaining: 26.8s
503:	learn: 0.0327323	total: 4.5s	remaining: 26.8s
504:	learn: 0.0326642	total: 4.51s	remaining: 26.8s
505:	learn: 0.0325430	total: 4.52s	remaining: 26.8s
506:	learn: 0.0325264	total: 4.53s	remaining: 26.8s
507:	learn: 0.0324100	total: 4.54s	remaining: 26.7s
508:	learn: 0.0323443	total: 4.55s	remaining: 26.7s
509:	learn: 0.0323125	total: 4.56s	remaining: 26.7s
510:	learn: 0.0322593	total: 4.57s	remaining: 26.7s
511:	learn: 0.0321980	total: 4.57s	remaining: 26.7s
512:	learn: 0.0321523	total: 4.58s	remaining: 26.7s
513:	learn: 0.

653:	learn: 0.0246512	total: 5.83s	remaining: 25.4s
654:	learn: 0.0245433	total: 5.84s	remaining: 25.4s
655:	learn: 0.0244784	total: 5.85s	remaining: 25.3s
656:	learn: 0.0244245	total: 5.85s	remaining: 25.3s
657:	learn: 0.0243652	total: 5.86s	remaining: 25.3s
658:	learn: 0.0243357	total: 5.87s	remaining: 25.3s
659:	learn: 0.0242534	total: 5.88s	remaining: 25.3s
660:	learn: 0.0241898	total: 5.89s	remaining: 25.3s
661:	learn: 0.0241880	total: 5.9s	remaining: 25.3s
662:	learn: 0.0241579	total: 5.91s	remaining: 25.3s
663:	learn: 0.0241089	total: 5.92s	remaining: 25.3s
664:	learn: 0.0240579	total: 5.92s	remaining: 25.3s
665:	learn: 0.0240137	total: 5.93s	remaining: 25.2s
666:	learn: 0.0239811	total: 5.94s	remaining: 25.2s
667:	learn: 0.0239466	total: 5.95s	remaining: 25.2s
668:	learn: 0.0239183	total: 5.96s	remaining: 25.2s
669:	learn: 0.0238920	total: 5.97s	remaining: 25.2s
670:	learn: 0.0238571	total: 5.98s	remaining: 25.2s
671:	learn: 0.0238087	total: 5.99s	remaining: 25.2s
672:	learn: 0

...

2689:	learn: 0.0013794	total: 24.9s	remaining: 7.49s
2690:	learn: 0.0013776	total: 24.9s	remaining: 7.48s
2691:	learn: 0.0013762	total: 24.9s	remaining: 7.47s
2692:	learn: 0.0013761	total: 24.9s	remaining: 7.46s
2693:	learn: 0.0013731	total: 24.9s	remaining: 7.45s
2694:	learn: 0.0013714	total: 24.9s	remaining: 7.44s
2695:	learn: 0.0013690	total: 24.9s	remaining: 7.43s
2696:	learn: 0.0013667	total: 24.9s	remaining: 7.42s
2697:	learn: 0.0013650	total: 24.9s	remaining: 7.41s
2698:	learn: 0.0013614	total: 24.9s	remaining: 7.4s
2699:	learn: 0.0013612	total: 25s	remaining: 7.39s
2700:	learn: 0.0013593	total: 25s	remaining: 7.38s
2701:	learn: 0.0013564	total: 25s	remaining: 7.37s
2702:	learn: 0.0013564	total: 25s	remaining: 7.37s
2703:	learn: 0.0013541	total: 25s	remaining: 7.36s
2704:	learn: 0.0013540	total: 25s	remaining: 7.35s
2705:	learn: 0.0013531	total: 25s	remaining: 7.34s
2706:	learn: 0.0013516	total: 25s	remaining: 7.33s
2707:	learn: 0.0013502	total: 25s	remaining: 7.32s
2708:	learn:

2860:	learn: 0.0011145	total: 26.4s	remaining: 5.91s
2861:	learn: 0.0011112	total: 26.5s	remaining: 5.9s
2862:	learn: 0.0011094	total: 26.5s	remaining: 5.89s
2863:	learn: 0.0011077	total: 26.5s	remaining: 5.88s
2864:	learn: 0.0011074	total: 26.5s	remaining: 5.87s
2865:	learn: 0.0011067	total: 26.5s	remaining: 5.86s
2866:	learn: 0.0011058	total: 26.5s	remaining: 5.85s
2867:	learn: 0.0011056	total: 26.5s	remaining: 5.84s
2868:	learn: 0.0011044	total: 26.5s	remaining: 5.83s
2869:	learn: 0.0011040	total: 26.5s	remaining: 5.82s
2870:	learn: 0.0011032	total: 26.5s	remaining: 5.81s
2871:	learn: 0.0011020	total: 26.5s	remaining: 5.8s
2872:	learn: 0.0011005	total: 26.5s	remaining: 5.79s
2873:	learn: 0.0010992	total: 26.6s	remaining: 5.78s
2874:	learn: 0.0010971	total: 26.6s	remaining: 5.78s
2875:	learn: 0.0010959	total: 26.6s	remaining: 5.76s
2876:	learn: 0.0010931	total: 26.6s	remaining: 5.76s
2877:	learn: 0.0010910	total: 26.6s	remaining: 5.75s
2878:	learn: 0.0010899	total: 26.6s	remaining: 5

3016:	learn: 0.0009176	total: 27.8s	remaining: 4.46s
3017:	learn: 0.0009166	total: 27.9s	remaining: 4.45s
3018:	learn: 0.0009153	total: 27.9s	remaining: 4.44s
3019:	learn: 0.0009134	total: 27.9s	remaining: 4.43s
3020:	learn: 0.0009127	total: 27.9s	remaining: 4.42s
3021:	learn: 0.0009116	total: 27.9s	remaining: 4.41s
3022:	learn: 0.0009110	total: 27.9s	remaining: 4.4s
3023:	learn: 0.0009099	total: 27.9s	remaining: 4.39s
3024:	learn: 0.0009079	total: 27.9s	remaining: 4.38s
3025:	learn: 0.0009072	total: 27.9s	remaining: 4.38s
3026:	learn: 0.0009047	total: 27.9s	remaining: 4.37s
3027:	learn: 0.0009036	total: 28s	remaining: 4.36s
3028:	learn: 0.0009023	total: 28s	remaining: 4.35s
3029:	learn: 0.0009022	total: 28s	remaining: 4.34s
3030:	learn: 0.0009009	total: 28s	remaining: 4.33s
3031:	learn: 0.0008995	total: 28s	remaining: 4.32s
3032:	learn: 0.0008994	total: 28s	remaining: 4.31s
3033:	learn: 0.0008987	total: 28s	remaining: 4.3s
3034:	learn: 0.0008985	total: 28s	remaining: 4.29s
3035:	learn

3189:	learn: 0.0007433	total: 29.4s	remaining: 2.86s
3190:	learn: 0.0007433	total: 29.4s	remaining: 2.85s
3191:	learn: 0.0007429	total: 29.4s	remaining: 2.84s
3192:	learn: 0.0007424	total: 29.5s	remaining: 2.83s
3193:	learn: 0.0007406	total: 29.5s	remaining: 2.82s
3194:	learn: 0.0007402	total: 29.5s	remaining: 2.81s
3195:	learn: 0.0007395	total: 29.5s	remaining: 2.8s
3196:	learn: 0.0007386	total: 29.5s	remaining: 2.79s
3197:	learn: 0.0007380	total: 29.5s	remaining: 2.79s
3198:	learn: 0.0007379	total: 29.5s	remaining: 2.78s
3199:	learn: 0.0007375	total: 29.5s	remaining: 2.77s
3200:	learn: 0.0007373	total: 29.5s	remaining: 2.76s
3201:	learn: 0.0007361	total: 29.5s	remaining: 2.75s
3202:	learn: 0.0007349	total: 29.6s	remaining: 2.74s
3203:	learn: 0.0007340	total: 29.6s	remaining: 2.73s
3204:	learn: 0.0007334	total: 29.6s	remaining: 2.73s
3205:	learn: 0.0007331	total: 29.6s	remaining: 2.72s
3206:	learn: 0.0007328	total: 29.7s	remaining: 2.71s
3207:	learn: 0.0007323	total: 29.7s	remaining: 

3357:	learn: 0.0006043	total: 31s	remaining: 1.31s
3358:	learn: 0.0006032	total: 31s	remaining: 1.3s
3359:	learn: 0.0006025	total: 31s	remaining: 1.29s
3360:	learn: 0.0006018	total: 31s	remaining: 1.28s
3361:	learn: 0.0006017	total: 31s	remaining: 1.27s
3362:	learn: 0.0006000	total: 31.1s	remaining: 1.26s
3363:	learn: 0.0005995	total: 31.1s	remaining: 1.26s
3364:	learn: 0.0005991	total: 31.1s	remaining: 1.25s
3365:	learn: 0.0005988	total: 31.1s	remaining: 1.24s
3366:	learn: 0.0005988	total: 31.1s	remaining: 1.23s
3367:	learn: 0.0005981	total: 31.1s	remaining: 1.22s
3368:	learn: 0.0005976	total: 31.1s	remaining: 1.21s
3369:	learn: 0.0005964	total: 31.1s	remaining: 1.2s
3370:	learn: 0.0005959	total: 31.1s	remaining: 1.19s
3371:	learn: 0.0005952	total: 31.1s	remaining: 1.18s
3372:	learn: 0.0005945	total: 31.1s	remaining: 1.17s
3373:	learn: 0.0005945	total: 31.2s	remaining: 1.16s
3374:	learn: 0.0005940	total: 31.2s	remaining: 1.15s
3375:	learn: 0.0005940	total: 31.2s	remaining: 1.15s
3376:

In [19]:
#(84,84,84,60,60,30,30,20,10,5,3)
nn = MLPRegressor(hidden_layer_sizes=(84,84,84,84,84,84,84,84,84), n_iter_no_change=200,
                  activation='relu', verbose=True,
                  learning_rate_init=.0001, tol=0.000001,
                  random_state=761, max_iter=30000,
                  alpha=.0000001, solver='adam',
                  learning_rate='adaptive')
nn.fit(X_train, y_train)

Iteration 1, loss = 38.41335018
Iteration 2, loss = 9.54206672
Iteration 3, loss = 2.16343009
Iteration 4, loss = 1.03469039
Iteration 5, loss = 0.80242602
Iteration 6, loss = 0.29602091
Iteration 7, loss = 0.13186006
Iteration 8, loss = 0.10839077
Iteration 9, loss = 0.07900698
Iteration 10, loss = 0.05924930
Iteration 11, loss = 0.04534687
Iteration 12, loss = 0.04113639
Iteration 13, loss = 0.03983982
Iteration 14, loss = 0.03919520
Iteration 15, loss = 0.03778860
Iteration 16, loss = 0.03677885
Iteration 17, loss = 0.03520073
Iteration 18, loss = 0.03438444
Iteration 19, loss = 0.03457884
Iteration 20, loss = 0.03396888
Iteration 21, loss = 0.03417207
Iteration 22, loss = 0.03391798
Iteration 23, loss = 0.03383107
Iteration 24, loss = 0.03357705
Iteration 25, loss = 0.03356773
Iteration 26, loss = 0.03332811
Iteration 27, loss = 0.03398540
Iteration 28, loss = 0.03396589
Iteration 29, loss = 0.03516348
Iteration 30, loss = 0.03350886
Iteration 31, loss = 0.03426195
Iteration 32, lo

Iteration 252, loss = 0.02496199
Iteration 253, loss = 0.02101527
Iteration 254, loss = 0.02058417
Iteration 255, loss = 0.02099340
Iteration 256, loss = 0.02100277
Iteration 257, loss = 0.02082414
Iteration 258, loss = 0.02114771
Iteration 259, loss = 0.02025040
Iteration 260, loss = 0.02025184
Iteration 261, loss = 0.02059488
Iteration 262, loss = 0.02251003
Iteration 263, loss = 0.02141913
Iteration 264, loss = 0.02122068
Iteration 265, loss = 0.02121416
Iteration 266, loss = 0.02036550
Iteration 267, loss = 0.02090169
Iteration 268, loss = 0.02139121
Iteration 269, loss = 0.02137034
Iteration 270, loss = 0.02358768
Iteration 271, loss = 0.02163120
Iteration 272, loss = 0.02115348
Iteration 273, loss = 0.02346583
Iteration 274, loss = 0.02207723
Iteration 275, loss = 0.02011719
Iteration 276, loss = 0.01965679
Iteration 277, loss = 0.01995914
Iteration 278, loss = 0.01980696
Iteration 279, loss = 0.02011290
Iteration 280, loss = 0.01952865
Iteration 281, loss = 0.02018988
Iteration 

Iteration 501, loss = 0.01763006
Iteration 502, loss = 0.01786463
Iteration 503, loss = 0.01829883
Iteration 504, loss = 0.01662275
Iteration 505, loss = 0.01629172
Iteration 506, loss = 0.01273355
Iteration 507, loss = 0.01458884
Iteration 508, loss = 0.01591691
Iteration 509, loss = 0.01999862
Iteration 510, loss = 0.01935401
Iteration 511, loss = 0.01527292
Iteration 512, loss = 0.01407755
Iteration 513, loss = 0.01365179
Iteration 514, loss = 0.02806469
Iteration 515, loss = 0.03276970
Iteration 516, loss = 0.02246352
Iteration 517, loss = 0.01776364
Iteration 518, loss = 0.01396533
Iteration 519, loss = 0.01224884
Iteration 520, loss = 0.01268066
Iteration 521, loss = 0.01260576
Iteration 522, loss = 0.01439568
Iteration 523, loss = 0.02129038
Iteration 524, loss = 0.02014452
Iteration 525, loss = 0.01510810
Iteration 526, loss = 0.01781177
Iteration 527, loss = 0.01464106
Iteration 528, loss = 0.01403371
Iteration 529, loss = 0.01347054
Iteration 530, loss = 0.01345397
Iteration 

Iteration 750, loss = 0.01493188
Iteration 751, loss = 0.01272819
Iteration 752, loss = 0.01114790
Iteration 753, loss = 0.00984711
Iteration 754, loss = 0.01493065
Iteration 755, loss = 0.01101481
Iteration 756, loss = 0.01111221
Iteration 757, loss = 0.01180321
Iteration 758, loss = 0.01301172
Iteration 759, loss = 0.01420937
Iteration 760, loss = 0.01136306
Iteration 761, loss = 0.00939452
Iteration 762, loss = 0.01401995
Iteration 763, loss = 0.01608824
Iteration 764, loss = 0.01621893
Iteration 765, loss = 0.01437971
Iteration 766, loss = 0.01669289
Iteration 767, loss = 0.01378731
Iteration 768, loss = 0.01041296
Iteration 769, loss = 0.01282545
Iteration 770, loss = 0.01354842
Iteration 771, loss = 0.01171660
Iteration 772, loss = 0.01063888
Iteration 773, loss = 0.00979669
Iteration 774, loss = 0.01119956
Iteration 775, loss = 0.01087281
Iteration 776, loss = 0.01442598
Iteration 777, loss = 0.01112758
Iteration 778, loss = 0.01080091
Iteration 779, loss = 0.01445145
Iteration 

Iteration 999, loss = 0.01087839
Iteration 1000, loss = 0.01357166
Iteration 1001, loss = 0.01331275
Iteration 1002, loss = 0.00902788
Iteration 1003, loss = 0.01187317
Iteration 1004, loss = 0.00843977
Iteration 1005, loss = 0.00860967
Iteration 1006, loss = 0.00903080
Iteration 1007, loss = 0.01057992
Iteration 1008, loss = 0.01595084
Iteration 1009, loss = 0.01227197
Iteration 1010, loss = 0.01023441
Iteration 1011, loss = 0.01911672
Iteration 1012, loss = 0.01489237
Iteration 1013, loss = 0.01611398
Iteration 1014, loss = 0.01380866
Iteration 1015, loss = 0.01047188
Iteration 1016, loss = 0.01107164
Iteration 1017, loss = 0.01844737
Iteration 1018, loss = 0.01787387
Iteration 1019, loss = 0.00953164
Iteration 1020, loss = 0.00834303
Iteration 1021, loss = 0.00799824
Iteration 1022, loss = 0.00985277
Iteration 1023, loss = 0.00827357
Iteration 1024, loss = 0.00971005
Iteration 1025, loss = 0.01245250
Iteration 1026, loss = 0.01195678
Iteration 1027, loss = 0.00892397
Iteration 1028,

...

Iteration 4127, loss = 0.00441208
Iteration 4128, loss = 0.00494161
Iteration 4129, loss = 0.00441201
Iteration 4130, loss = 0.00400121
Iteration 4131, loss = 0.00463149
Iteration 4132, loss = 0.00452940
Iteration 4133, loss = 0.00441589
Iteration 4134, loss = 0.00584822
Iteration 4135, loss = 0.00479675
Iteration 4136, loss = 0.00486454
Iteration 4137, loss = 0.00603395
Iteration 4138, loss = 0.00658567
Iteration 4139, loss = 0.00968844
Iteration 4140, loss = 0.00871215
Iteration 4141, loss = 0.00864096
Iteration 4142, loss = 0.00598625
Iteration 4143, loss = 0.00823393
Iteration 4144, loss = 0.00638269
Iteration 4145, loss = 0.00435406
Iteration 4146, loss = 0.00583220
Iteration 4147, loss = 0.00426518
Iteration 4148, loss = 0.00417113
Iteration 4149, loss = 0.00418788
Iteration 4150, loss = 0.00421018
Iteration 4151, loss = 0.00567114
Iteration 4152, loss = 0.00406397
Iteration 4153, loss = 0.00417330
Iteration 4154, loss = 0.00432430
Iteration 4155, loss = 0.00376739
Iteration 4156

Iteration 4367, loss = 0.00402007
Iteration 4368, loss = 0.00373846
Iteration 4369, loss = 0.00365689
Iteration 4370, loss = 0.00406793
Iteration 4371, loss = 0.00586692
Iteration 4372, loss = 0.00590472
Iteration 4373, loss = 0.00451680
Iteration 4374, loss = 0.00371532
Iteration 4375, loss = 0.00412707
Iteration 4376, loss = 0.00448635
Iteration 4377, loss = 0.00388451
Iteration 4378, loss = 0.00436299
Iteration 4379, loss = 0.00525245
Iteration 4380, loss = 0.00618360
Iteration 4381, loss = 0.00576439
Iteration 4382, loss = 0.00470377
Iteration 4383, loss = 0.00429479
Iteration 4384, loss = 0.00370964
Iteration 4385, loss = 0.00364818
Iteration 4386, loss = 0.00361510
Iteration 4387, loss = 0.00400077
Iteration 4388, loss = 0.00403362
Iteration 4389, loss = 0.00373454
Iteration 4390, loss = 0.00434591
Iteration 4391, loss = 0.00395960
Iteration 4392, loss = 0.00508538
Iteration 4393, loss = 0.00657685
Iteration 4394, loss = 0.00454525
Iteration 4395, loss = 0.00474720
Iteration 4396

Iteration 4608, loss = 0.00384375
Iteration 4609, loss = 0.00394017
Iteration 4610, loss = 0.00358153
Iteration 4611, loss = 0.00361585
Iteration 4612, loss = 0.00380770
Iteration 4613, loss = 0.00378442
Iteration 4614, loss = 0.00382537
Iteration 4615, loss = 0.00371701
Iteration 4616, loss = 0.00390294
Iteration 4617, loss = 0.00510408
Iteration 4618, loss = 0.00424779
Iteration 4619, loss = 0.00420945
Iteration 4620, loss = 0.00420773
Iteration 4621, loss = 0.00371249
Iteration 4622, loss = 0.00381860
Iteration 4623, loss = 0.00512097
Iteration 4624, loss = 0.00383214
Iteration 4625, loss = 0.00394074
Iteration 4626, loss = 0.00365985
Iteration 4627, loss = 0.00349204
Iteration 4628, loss = 0.00401605
Iteration 4629, loss = 0.00390732
Iteration 4630, loss = 0.00362505
Iteration 4631, loss = 0.00347371
Iteration 4632, loss = 0.00354013
Iteration 4633, loss = 0.00373083
Iteration 4634, loss = 0.00365780
Iteration 4635, loss = 0.00393356
Iteration 4636, loss = 0.00412774
Iteration 4637

Iteration 4849, loss = 0.00336515
Iteration 4850, loss = 0.00377764
Iteration 4851, loss = 0.00463993
Iteration 4852, loss = 0.00507929
Iteration 4853, loss = 0.00359652
Iteration 4854, loss = 0.00434327
Iteration 4855, loss = 0.00449482
Iteration 4856, loss = 0.00388736
Iteration 4857, loss = 0.00496893
Iteration 4858, loss = 0.00383389
Iteration 4859, loss = 0.00541496
Iteration 4860, loss = 0.00655746
Iteration 4861, loss = 0.00468109
Iteration 4862, loss = 0.00436644
Iteration 4863, loss = 0.00388761
Iteration 4864, loss = 0.00488905
Iteration 4865, loss = 0.00367909
Iteration 4866, loss = 0.00375012
Iteration 4867, loss = 0.00395203
Iteration 4868, loss = 0.00375882
Iteration 4869, loss = 0.00345089
Iteration 4870, loss = 0.00341472
Iteration 4871, loss = 0.00422570
Iteration 4872, loss = 0.00376427
Iteration 4873, loss = 0.00358243
Iteration 4874, loss = 0.00428541
Iteration 4875, loss = 0.00388711
Iteration 4876, loss = 0.00351635
Iteration 4877, loss = 0.00346212
Iteration 4878

MLPRegressor(alpha=1e-07,
             hidden_layer_sizes=(84, 84, 84, 84, 84, 84, 84, 84, 84),
             learning_rate='adaptive', learning_rate_init=0.0001,
             max_iter=30000, n_iter_no_change=200, random_state=761, tol=1e-06,
             verbose=True)

In [20]:
y_pred_model = rf.predict(X_test)
print('RMSE RF: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_model)))

y_pred_model = svr.predict(X_test)
print('RMSE SVR: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_model)))

y_pred_model = xgb.predict(X_test)
print('RMSE XGB: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_model)))

y_pred_model = abr.predict(X_test)
print('RMSE ABR: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_model)))

y_pred_model = lgb.predict(X_test)
print('RMSE LGB: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_model)))

y_pred_model = knn.predict(X_test)
print('RMSE KNN: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_model)))

y_pred_model = nn.predict(X_test)
print('RMSE NN: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_model)))

y_pred_model = cat.predict(X_test)
print('RMSE CAT: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_model)))

y_pred_model = gbr.predict(X_test)
print('RMSE GBR: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_model)))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 632 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 997 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1442 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1500 out of 1500 | elapsed:    0.4s finished


RMSE RF: 0.131281
RMSE SVR: 0.114623
RMSE XGB: 0.118756
RMSE ABR: 0.137749
RMSE LGB: 0.117895
RMSE KNN: 0.187958
RMSE NN: 0.128531
RMSE CAT: 0.111504
RMSE GBR: 0.111460


In [ ]:
level0 = [('rf',rf),('xgb',xgb),('gbr',gbr),('lgb',lgb),
          ('abr',abr),('cat',cat),('svr',svr),('nn',nn),
         ('knn',knn)]
level1 = xgb = XGBRegressor(booster='gbtree',
                   colsample_bylevel=0.6, colsample_bynode=.4,#1,.5
                   colsample_bytree=0.6, gamma=0,
                   importance_type='gain', learning_rate=0.01,
                   max_delta_step=0, max_depth=45,
                   min_child_weight=4, n_estimators=2500,
                   n_jobs=-1, nthread=None,
                   objective='reg:linear', reg_alpha=0.6,
                   reg_lambda=0.6, scale_pos_weight=1,
                   silent=None, subsample=0.8,
                   verbosity=2)
#LinearRegression()
stack = StackingRegressor(estimators=level0,final_estimator=level1,
                          cv=5, verbose=2, n_jobs=-1)
stack.fit(X_train, y_train)
y_pred_stack= stack.predict(X_test)
print('RMSE: %2f'% np.sqrt(mean_squared_error(y_test, y_pred_stack)))

In [ ]:
predict_test_data = stack.predict(X_test_house)
predict_test_data

In [ ]:
sub_result = pd.DataFrame()
test = pd.DataFrame(test_csv)
sub_result['Id'] = test['Id']
sub_result['SalePrice'] = np.expm1(predict_test_data)
sub_result

In [ ]:
sub_result.to_csv('submission.csv', index=False)